In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
from datetime import datetime
from hyperparams import Hyperparams

In [2]:
def encode(day_of_year, max_val):
    sin1 = np.sin(2  * np.pi * day_of_year/max_val)
    sin2 = np.sin(4  * np.pi * day_of_year/max_val)
    sin3 = np.sin(6  * np.pi * day_of_year/max_val)
    sin4 = np.sin(8  * np.pi * day_of_year/max_val)
    
    cos1 = np.cos(2 * np.pi * day_of_year/max_val)
    cos2 = np.cos(4 * np.pi * day_of_year/max_val)
    cos3 = np.cos(6 * np.pi * day_of_year/max_val) 
    cos4 = np.cos(8 * np.pi * day_of_year/max_val)
    
    return np.array([sin1, sin2, sin3, sin4, cos1, cos2, cos3, cos4])

In [3]:
data_frame=pd.read_csv('chicago15.csv',index_col=0)

In [4]:
times=data_frame.index

In [5]:
num_covariates=26

In [6]:
    covariates = np.zeros((times.shape[0], num_covariates))
    weights = np.zeros((times.shape[0], num_covariates-24)) 
    #Trừ 24 là vì mấy cái thời gian (calendar factors) không có trọng số
    #Lưu ý: Paul trừ tới 26 vì Paul còn 2 cột liên quan đến holiday và ngày nghỉ của tàu 
    
    day_of_week = np.zeros((times.shape[0]))
    weekend = np.zeros((times.shape[0]))
    # Làm việc với thời gian    
    times_df = times.to_frame(index=False)
    times_df['time'] = pd.to_datetime(times_df['time']) #trích giờ
    times_df['date'] = times_df['time'].astype(str).str[:10] #trích ra ngày
    #Trích tiếng
    times_df['hr'] =  round((times_df['time'].dt.hour.replace(to_replace=0, value=24) + times_df['time'].dt.minute/60),1)
    #Trích tiếng mà không lấy phần phút để lát ghép meteo vào 
    times_df['hrr'] =  times_df['time'].dt.hour

In [7]:
def gen_covariates(times, num_covariates):
    covariates = np.zeros((times.shape[0], num_covariates))
    weights = np.zeros((times.shape[0], num_covariates-24)) 
    #Trừ 24 là vì mấy cái thời gian (calendar factors) không có trọng số
    #Lưu ý: Paul trừ tới 26 vì Paul còn 2 cột liên quan đến holiday và ngày nghỉ của tàu 
    
    day_of_week = np.zeros((times.shape[0]))
    weekend = np.zeros((times.shape[0]))
    # Làm việc với thời gian    
    times_df = times.to_frame(index=False)
    times_df['time'] = pd.to_datetime(times_df['time']) #trích giờ
    times_df['date'] = times_df['time'].astype(str).str[:10] #trích ra ngày
    #Trích tiếng
    times_df['hr'] =  round((times_df['time'].dt.hour.replace(to_replace=0, value=24) + times_df['time'].dt.minute/60),1)
    #Trích tiếng mà không lấy phần phút để lát ghép meteo vào 
    times_df['hrr'] =  times_df['time'].dt.hour

    for i, input_time in enumerate(times):
        #Chuyển đổi thành định dạng date time
        input_time = datetime.strptime(input_time, '%m/%d/%Y %H:%M')
        #Trích ra ngày trong năm
        day_of_year = input_time.timetuple().tm_yday
        #Tiến hành mã hóa ngày trong THÁNG vì dữ liệu chúng ta chỉ nằm trong Tháng => Nếu trong năm thì sửa lại :) 
        covariates[i, 1:9] = encode(day_of_year, 31) #Vì tháng 31 có 31 ngày :)))) 

        hour = input_time.hour #trích giờ ra
        if hour == 0: 
            hour = 24
        timestep = (hour + input_time.minute/60)*2 -1  
        covariates[i, 9:17] = encode(timestep, 96) #Encode POSITON OF THE DAY
        day_of_week[i] = input_time.weekday() 
        if input_time.weekday() >= 5:  # 5 là thứ 7, 6 là chủ nhật
            weekend[i] = 1
    covariates[:, 17:24] = pd.get_dummies(pd.Series(day_of_week)).to_numpy() #One hot encode cho ngày trong tuần
    #Tới đây đã mã hóa thành công: Ngày trong THÁNG, position of ngày, day of week (one hot encode)
    #Bây giờ sẽ mã hóa "Week day" và "Weekend" nhưng hơi confused về việc nó có cần thiết không ? vì nó đã có trong day of week rồi mà ta
    # Thảo luận !!!
    #covariates[:,24] = weekend
    ########Meteo
    #Trích dữ liệu lấy tại 3 cột temperature, precipitation 3h
    meteo = (pd.read_csv('chicagometeo.csv', sep=",", parse_dates=True, decimal='.'))[['date', 'hrr','temp','precip']]  
    meteo['date'] = pd.to_datetime(meteo['date']) #ĐOẠN NÀY TUI TỰ SỬA HHIHIHIHIHIIH >"<
    meteo['date'] = meteo['date'].apply(lambda x: x.strftime('%Y-%m-%d')) #PHẢI CÓ !! NẾU KHÔNG NÓ LỌC SAI BÉT HAHAHA
    times_df = pd.merge(times_df, meteo, how='left', left_on = ['date', 'hrr'], right_on = ['date', 'hrr'])
    #Kiểm tra xem có NAN trong 2 cột "temperature" và "precipitations3h" hay không? Nếu có thì thay bằng 0
    for met in ['temp','precip']:
        times_df[met] = np.where(times_df[met].isna(), 0, times_df[met])   
    #Đưa vào bảng covariates và chuẩn hóa
    covariates[:, 24:26] = times_df[['temp', 'precip']].to_numpy()
    covariates[:,24:26] = covariates[:,24:26]/(covariates[:, 24:26]).max(0)
    #Tính trọng số
    w_meteo = np.absolute(times_df[['temp', 'precip']].to_numpy())
    weights[:,0:2] = w_meteo
    return covariates[:, :num_covariates], weights
    
    
    

In [8]:
covariates, weights = gen_covariates(data_frame.index, num_covariates=26)

In [9]:
covariates

array([[ 0.        , -0.20129852, -0.39435586, ...,  0.        ,
         0.14159292,  0.        ],
       [ 0.        , -0.20129852, -0.39435586, ...,  0.        ,
         0.14159292,  0.        ],
       [ 0.        , -0.20129852, -0.39435586, ...,  0.        ,
         0.14159292,  0.        ],
       ...,
       [ 0.        , -0.39435586, -0.72479279, ...,  1.        ,
         0.26548673,  0.07156168],
       [ 0.        , -0.39435586, -0.72479279, ...,  1.        ,
         0.26548673,  0.07156168],
       [ 0.        , -0.39435586, -0.72479279, ...,  1.        ,
         0.26548673,  0.07156168]])

In [10]:
covariates = pd.DataFrame(covariates)
covariates = covariates.set_index(data_frame.index)

In [11]:
covariates

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
time,,,,,,,,,,,,,,,,,,,,,
3/1/2024 0:00,0.0,-0.201299,-0.394356,-0.571268,-0.724793,0.979530,0.918958,0.820763,0.688967,6.540313e-02,...,0.965926,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.141593,0.000000
3/1/2024 0:15,0.0,-0.201299,-0.394356,-0.571268,-0.724793,0.979530,0.918958,0.820763,0.688967,3.271908e-02,...,0.991445,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.141593,0.000000
3/1/2024 0:30,0.0,-0.201299,-0.394356,-0.571268,-0.724793,0.979530,0.918958,0.820763,0.688967,1.224647e-16,...,1.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.141593,0.000000
3/1/2024 0:45,0.0,-0.201299,-0.394356,-0.571268,-0.724793,0.979530,0.918958,0.820763,0.688967,-3.271908e-02,...,0.991445,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.141593,0.000000
3/1/2024 1:00,0.0,-0.201299,-0.394356,-0.571268,-0.724793,0.979530,0.918958,0.820763,0.688967,6.540313e-02,...,0.965926,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.119469,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3/31/2024 22:45,0.0,-0.394356,-0.724793,-0.937752,-0.998717,0.918958,0.688967,0.347305,-0.050649,2.270763e-01,...,0.608761,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.225664,0.313082
3/31/2024 23:00,0.0,-0.394356,-0.724793,-0.937752,-0.998717,0.918958,0.688967,0.347305,-0.050649,1.950903e-01,...,0.707107,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.265487,0.071562
3/31/2024 23:15,0.0,-0.394356,-0.724793,-0.937752,-0.998717,0.918958,0.688967,0.347305,-0.050649,1.628955e-01,...,0.793353,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.265487,0.071562


In [12]:
def date_range_list(start_date, end_date):
    # Return list of datetime.date objects between start_date and end_date (inclusive).
    date_list = []
    curr_date = start_date
    while curr_date <= end_date:
        date_list.append(curr_date)
        curr_date += timedelta(days=1)
    return date_list

In [13]:
params = Hyperparams(hybridize=True)
empty_weights =  np.zeros(shape=(params.pred_days*1*96,1)) 

In [14]:
def create_weights(w,data):
    time_len = data.shape[0] #Tất cả mốc tgian
    params = Hyperparams(hybridize=True) #Lấy params
    stride_size = 96 #Dự đoán 24 điểm tiếp theo 
    window_size = params.pred_days*2*stride_size
    input_size = window_size-stride_size
    total_windows = (time_len-input_size) 
    
    weight_indic = np.zeros((total_windows, 1), dtype='float32')   

    count = 0
    for i in range(input_size,total_windows):
        window_start = i - input_size
        window_end = i

        weight_indic[count] = (np.round(w[window_start:window_end, :].sum())).astype(int)
        count += 1
    return weight_indic

In [15]:
weight_indic = pd.DataFrame(np.concatenate((empty_weights,create_weights(weights,data_frame)), axis=0))
weight_indic = weight_indic.set_index(data_frame.index)

In [16]:
weight_indic

,0
time,
3/1/2024 0:00,0.0
3/1/2024 0:15,0.0
3/1/2024 0:30,0.0
3/1/2024 0:45,0.0
3/1/2024 1:00,0.0
...,...
3/31/2024 22:45,0.0
3/31/2024 23:00,0.0
3/31/2024 23:15,0.0


In [17]:
weight_indic.to_csv('weight_indic_chicago15.csv')

In [18]:
covariates.to_csv('ChicagoCovariates15.csv')